In [1]:
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import ParameterGrid
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import metrics
from notebooks.sentiment.util import eval
from sentiment.new_data import InterTASSAugmented
from sentiment.tass import InterTASSReader
from tqdm import tqdm

def preprocess_tweets(str_):
    mentions = r'(?:@[^\s]+)'
    urls = r'(?:https?\://t.co/[\w]+)'
    str_ = re.sub(mentions, '', str_)
    return re.sub(urls, '', str_)

param_grid = {
    'clf__random_state': [0],
    'clf__penalty': ('l2', 'l2'),
    'clf__C': (0.1, 0.3, 0.5, 0.9),
}

vect = TfidfVectorizer(tokenizer=word_tokenize,
                       binary=True,
                       analyzer="char_wb",
                       ngram_range=(1, 6),
                       min_df=5,
                       max_df=0.95,
                       preprocessor=preprocess_tweets,
                       stop_words=stopwords.words("spanish"))

vect2 = TfidfVectorizer(tokenizer=word_tokenize,
                        binary=True,
                        analyzer="word",
                        ngram_range=(1, 5),
                        preprocessor=preprocess_tweets,
                        stop_words=stopwords.words("spanish"))

params_list = list(ParameterGrid(param_grid))

In [4]:
pipeline = Pipeline([
            ('feats', FeatureUnion([
                    ('vect', vect),  # can pass in either a pipeline
                    ('vect1', vect2),  # or a transformer
                ])),
            ('clf', LogisticRegression()),
        ])
reader = InterTASSAugmented(ratio=0.9)  # Class to use augmented data
X_train, y_train = reader.Xy()
# train = "intertass-ES-train-tagged.xml"
# train = InterTASSReader(train)
# X_train, y_train = list(train.X()), list(train.y())


corpus = "../intertass-ES-development-tagged.xml"
dev = InterTASSReader(corpus)
X_dev, y_dev = list(dev.X()), list(dev.y())

results = []
for params in tqdm(params_list):
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    result = eval(pipeline, X_dev, y_dev)
    
    results.append({
        **result,
        **params,
    })

  0%|          | 0/8 [00:00<?, ?it/s]/home/gosen/.local/share/virtualenvs/PLN-2019-N1iczKTA/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gosen/.local/share/virtualenvs/PLN-2019-N1iczKTA/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/gosen/.local/share/virtualenvs/PLN-2019-N1iczKTA/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|██████████| 8/8 [00:15<00:00,  1.96s/it]


In [5]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)[:10]

,acc,clf__C,clf__penalty,clf__random_state,f1
4,0.604743,0.5,l2,0,0.390455
5,0.604743,0.5,l2,0,0.390455
6,0.598814,0.9,l2,0,0.403118
7,0.598814,0.9,l2,0,0.403118
2,0.577075,0.3,l2,0,0.339400
3,0.577075,0.3,l2,0,0.339400
0,0.537549,0.1,l2,0,0.292038
1,0.537549,0.1,l2,0,0.292038
